In [1]:
import warnings
from pandas import read_csv
from pandas import datetime
from pandas import DataFrame
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
series1 = read_csv('PredictTea-train-edit5.csv', header=0, index_col=0, squeeze=True)
series2 = read_csv('PredictTea-test-edit5.csv', header=0, index_col=0, squeeze=True)

In [3]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(train_dataset, test_dataset, arima_order):
	history = [x for x in train_dataset]
	predictions = list()
	#make predictions
	for t in range(len(test_dataset)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit(disp=0)
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test_dataset[t])
	# calculate out of sample error
	error = mean_squared_error(test_dataset, predictions)
	return error

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(train_dataset, test_dataset, p_values, d_values, q_values):
	#train_dataset = train_dataset.tolist()
	#test_dataset = test_dataset.tolist()
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					rmse = sqrt(evaluate_arima_model(train_dataset, test_dataset, order))                 
					if rmse < best_score:
						best_score, best_cfg = rmse, order
					print('ARIMA%s MSE=%.3f' % (order,rmse))
				except:
					continue
	print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))
    
# evaluate parameters
p_values = range(0, 3)
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(series1.values, series2.values, p_values, d_values, q_values)

ARIMA(0, 0, 0) MSE=0.224
ARIMA(0, 0, 1) MSE=0.204
ARIMA(0, 0, 2) MSE=0.207
ARIMA(0, 1, 0) MSE=0.215
ARIMA(0, 1, 1) MSE=0.217
ARIMA(0, 1, 2) MSE=0.205
ARIMA(0, 2, 0) MSE=0.345
ARIMA(0, 2, 1) MSE=0.218
ARIMA(1, 0, 0) MSE=0.192
ARIMA(1, 0, 1) MSE=0.208
ARIMA(1, 1, 0) MSE=0.209
ARIMA(1, 1, 1) MSE=0.192
ARIMA(1, 2, 0) MSE=0.273
ARIMA(2, 0, 0) MSE=0.199
ARIMA(2, 0, 1) MSE=0.219
ARIMA(2, 1, 0) MSE=0.220
ARIMA(2, 1, 1) MSE=0.198
ARIMA(2, 2, 0) MSE=0.282
Best ARIMA(1, 0, 0) MSE=0.192
